<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Clustering the Explanations of the Metr-LA dataset</h2>
</center>

---

In this notebook the important subgraphs obtained by the *Explainer* on the *Metr-LA* dataset are clustered in order to subdivide it in meaningful events such as congestions or free-flows.

Firstly, a distance matrix is obtained for each important subgraph of the datasets in order to compute the spatio-temporal and speed distance among all nodes in the prediction. The distance matrix is obtained throught the method explained in *Revealing the day-to-day regularity of urban congestion patterns with
3d speed maps* <a name="cite_paper"></a>[<sup>[1]</sup>](#note_paper)

Finally, the important subgraphs are clustered through *Agglomerative Clustering*, while considering the distance matrix as a dissimilarity measure.

For more detailed informations about the used functions, look into the corresponding docstrings inside the python files, inside the `src` folder.

---
<small>

<a name="note_paper"></a>[1] 
C. Lopez et al. “Revealing the day-to-day regularity of urban congestion patterns with
3d speed maps”. In: *Scientific Reports, 7(1):14029*, September 2017. ISSN:
2045-2322. DOI: 10.1038/s41598-017-14237-8. URL: https://doi.org/10.1038/s41598-017-14237-8.
</small>

In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

# 1 Loading the Data
In this section the data is loaded.

In [4]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [5]:
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

In [6]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [7]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [8]:
import os
import numpy as np
from src.spatial_temporal_gnn.prediction import predict

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_train.npy'))[..., :1]
x_val = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_val.npy'))[..., :1]
x_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))[..., :1]

The speeds of the important subgraphs are turned into km/h.

In [9]:
from src.utils.config import MPH_TO_KMH_FACTOR

# Turn the dataset in kilometers per hour.
x_train = x_train * MPH_TO_KMH_FACTOR
x_val = x_val * MPH_TO_KMH_FACTOR
x_test = x_test * MPH_TO_KMH_FACTOR

In [10]:
_, n_timesteps, n_nodes, _ = x_train.shape

# 2 Distance matrix
The distance matrix $M$ used as a metric of dissimilarity in Agglomerative Clustering is computed separately for each instance and is composed of:
* A *spatial distance matrix* $M_d$;
* A *temporal distance matrix* $M_t$;
* A *speed distance matrix* $M_s$

The matrices $M_d$, $M_t$ and $M_s$ are each scaled through *min-max scaling* between $0$ and $1$ and summed together in order to obtain $M$ as follows: 
$$M = W \cdot M_s + M_d + M_t$$
where the speed distance is overweighted by multiplying $M_s$ by a factor $W \geq 1$ because the speed variable is expected to play a predominant role during the clustering process. $M$ is next normalized again between $0$ and $1$ through min-max scaling.

## 2.1 Spatial Distance Matrix
The *spatial distance matrix* $M_d$ is an $(N \cdot T') \times (N \cdot T')$ matrix derived from the adjacency matrix of the traffic network $A \in \mathbb{R}^{N \times N}$ and describing the spatial distant of each nodes regardless of their timestep.

In [12]:
from src.explanation.clustering.clustering import (
    get_adjacency_distance_matrix)

adj_distance_matrix = get_adjacency_distance_matrix(adj_matrix, n_timesteps)

In [14]:
print(f'Shape of the Adjacency Distance Matrix: {adj_distance_matrix.shape}')

Shape of the Adjacency Distance Matrix: (2484, 2484)


# 2.2 Temporal Distance Matrix
The *temporal distance matrix* $M_t$ is an $(N \cdot T') \times (N \cdot T')$ matrix describing the temporal distance of the nodes at each timestep.

In [15]:
from src.explanation.clustering.clustering import (
    get_temporal_distance_matrix)

temporal_distance_matrix = get_temporal_distance_matrix(n_nodes, n_timesteps)

In [16]:
print('Shape of the Temporal Distance Matrix:',
      f'{temporal_distance_matrix.shape}')

Shape of the Temporal Distance Matrix: (2484, 2484)


# 2.3 Speed Distance Matrix
The *speed distance matrix* $M_s$ is an $(N \cdot T') \times (N \cdot T')$ matrix describing the speed distance of the nodes at each timestep.

It is computed for each instance separately, before Agglomerative Clustering is performed. The value $W$, overweighting it is set as $3$.

# 3 Clustering Function
Next, the clustering technique *Agglomerative Clustering* is used on the distance matrix $M$. It identifies whether each node of the important subgraph, specified by an index in $M$, is part of a distinct traffic cluster.

The number of clusters $n$ is tested on a range $[1, 5]$ and $n$ leading to the best score is selected.

Score is computed as the ratio between *Cluster Dissimilarity* and *Within Cluster Variance* in order to favor dissimilar clusters with low variance:
$$ Score = \frac{CD}{WCV} $$

Within Cluster Variance and Cluster Dissimilarity are computed as follows:
* **Within Cluster Variance:**
    $$WCV = \frac{1}{\sum_{i = 1}^N n_i} \frac{\sum_{i = 1}^N n_i \sigma^2_i}{\sigma^2} $$
    with $N$ the number of clusters, $n_i$ the nodes of the $i^{th}$ cluster in the predicted network and $\sigma_i^2$ the speed variance among its nodes. $\sigma^2$ is the variance among all nodes of the prediction.

* **Cluster Dissimilarity:**
    $$ CD = \frac{\sum_{i = 1}^N \sum_{k = i + 1}^N \sqrt{n_i \cdot n_k} \cdot |\mu_i - \mu_k|}{\sum_{i = 1}^N \sum_{k = i + 1}^N \sqrt{n_i \cdot n_k}} $$
    with $\mu_i$ the mean speed value of cluster $i$.

The results are then saved for the train, validation and test sets.

In [160]:
DATA_DIR = os.path.join(BASE_DATA_DIR, 'clustered')

os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
from src.explanation.clustering.clustering_explanations import get_explanation_clusters_dataset

x_train_clustered = get_explanation_clusters_dataset(
    x_train,
    adj_distance_matrix,
    temporal_distance_matrix)

np.save(os.path.join(DATA_DIR, 'x_train.npy'), x_train_clustered)

x_val_clustered = get_explanation_clusters_dataset(
    x_val,
    adj_distance_matrix,
    temporal_distance_matrix)

np.save(os.path.join(DATA_DIR, 'x_val.npy'), x_val_clustered)

x_test_clustered = get_explanation_clusters_dataset(
    x_test,
    adj_distance_matrix,
    temporal_distance_matrix)

np.save(os.path.join(DATA_DIR, 'x_test.npy'), x_test_clustered)